In [1]:
import h5py
import numpy as np

In [2]:
from ctapipe.instrument import CameraGeometry
from ctapipe.image.geometry_converter import convert_geometry_hex1d_to_rect2d
from ctapipe.calib import CameraCalibrator
from ctapipe.io import event_source
from ctapipe.utils import get_dataset

In [3]:
filename = get_dataset_path('gamma_test_large.simtel.gz')
source = event_source(filename)
calib = CameraCalibrator()

In [4]:
h5f = h5py.File('images.h5', 'w')
maxevents = 1 # number of events to save in hdf5 file
count = 1

In [5]:
try:
    for event in source:
        
        count +=1
        
        # calibrating images
        calib.calibrate(event)

        # fill hdf5 with structure
        # telescope groups
        #
        for tel_id, dl1cameracontainer in event.dl1.tel.items():
            telname = "tel_{:03d}".format(tel_id)

            # camera channels groups
            #
            for chan in range(dl1cameracontainer.image.shape[0]):
                channelname = "chan_{:02}".format(chan)

                # create images with camera geometry
                #
                camgeom = event.inst.tel[tel_id].camera

                signal = dl1cameracontainer.image[chan]
                new_geom, image2d = convert_geometry_hex1d_to_rect2d(camgeom, signal)
        
                # write images in each channel group
                #
                #2d image
                imagename = '/DL1/'+telname+'/'+channelname+'/image'+str(event.r0.event_id)      
                image5f = h5f.create_dataset(imagename, data=image2d)

                # set image attributes
                image5f.attrs['CLASS'] = 'IMAGE'
                image5f.attrs['IMAGE_VERSION'] = '1.2'
                image5f.attrs['IMAGE_MINMAXRANGE'] = [np.nanmin(signal), np.nanmax(signal)]

                print(imagename)
                
                #3d extracted_samples
                #2d peakpos
                #3d cleaned
        
        if count>maxevents: break
            
finally:
   h5f.close()                  

/DL1/tel_025/chan_00/image23703
/DL1/tel_034/chan_00/image23703
/DL1/tel_034/chan_01/image23703
/DL1/tel_021/chan_00/image23703
/DL1/tel_013/chan_00/image23703


# Generated HDF5 File

* [DL1 Images File](images.h5)